# Consistency Checks: Reading Exercises

## Contains:
### Descriptive statistics for orders
### Practice with mixed data types on sample dataframe
### Identify missing values in products
### Subset with missing values removed from products
### Find and remove full duplicates from products
### Check for mixed datatypes in orders
### Check for missing values in orders
### Investigation of missing values all have order_number of 1
### Impute missing values with mean days since last order of 11
### Check orders for duplicates

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

In [2]:
#create file path
path = r'C:\Users\krist\08.2023_InstacartBasketAnalysis'

In [3]:
#import products 
df_prods = pd.read_csv(os.path.join(path, 
                                    '02.Data', 
                                    'Original_Data', 
                                    'products.csv'))
df_prods.head()

,product_id,product_name,aisle_id,department_id,prices
0,1,Chocolate Sandwich Cookies,61,19,5.8
1,2,All-Seasons Salt,104,13,9.3
2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,5,Green Chile Anytime Sauce,5,13,4.3


In [8]:
#import orders_wrangled
df_ords = pd.read_csv(os.path.join(path, 
                                   '02.Data', 
                                   'Prepared_Data', 
                                   'orders_wrangled.csv'), 
                      index_col = False)
df_ords.head()

,Unnamed: 0,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0


In [9]:
#get rid of extraneous index column 'Unnamed: 0'
df_ords = df_ords.drop(columns = ['Unnamed: 0'])
df_ords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3421083 entries, 0 to 3421082
Data columns (total 6 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   order_id               int64  
 1   user_id                int64  
 2   order_number           int64  
 3   order_day_of_week      int64  
 4   order_hour_of_day      int64  
 5   days_since_last_order  float64
dtypes: float64(1), int64(5)
memory usage: 156.6 MB


In [10]:
df_ords.describe()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


## Create a dataframe to practice mixed data types

In [14]:
#create practice data frame note that DataFrame is in camel case
df_test = pd.DataFrame()

In [16]:
#create mixed type column named mix
df_test['mix'] = ['a', 'b', 1, True]
df_test.head()

,mix
0,a
1,b
2,1
3,True


In [18]:
#check for mixed types, this will print the column name of any column with mixed data types
for col in df_test.columns.tolist():
  weird = (df_test[[col]].applymap(type) != df_test[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_test[weird]) > 0:
    print (col)

mix


In [19]:
#change datatype to string for consistency 
df_test['mix'] = df_test['mix'].astype('str')

## Finding missing values

In [20]:
df_prods.isnull().sum()

product_id        0
product_name     16
aisle_id          0
department_id     0
prices            0
dtype: int64

### product_name has 16 missing values

In [22]:
#create a subset to view all observations with missing values in product_name
df_nan = df_prods[df_prods['product_name'].isnull() == True]
df_nan

,product_id,product_name,aisle_id,department_id,prices
33,34,NaN,121,14,12.2
68,69,NaN,26,7,11.8
115,116,NaN,93,3,10.8
261,262,NaN,110,13,12.1
525,525,NaN,109,11,1.2
1511,1511,NaN,84,16,14.3
1780,1780,NaN,126,11,12.3
2240,2240,NaN,52,1,14.2
2586,2586,NaN,104,13,12.4
3159,3159,NaN,126,11,13.1


In [23]:
#check the total number of rows in products to compare to subset and see if removal is an option
df_prods.shape

(49693, 5)

### 16 out of 49,693 rows have missing values 

In [24]:
#create new subset with missing values removed 
df_prods_clean = df_prods[df_prods['product_name'].isnull() == False]

In [25]:
#check to make sure missing values removed 
df_prods_clean.shape

(49677, 5)

## Finding Duplicates

In [26]:
#look for duplicates in products
df_dups = df_prods_clean[df_prods_clean.duplicated()]
df_dups

,product_id,product_name,aisle_id,department_id,prices
462,462,Fiber 4g Gummy Dietary Supplement,70,11,4.8
18459,18458,Ranger IPA,27,5,9.2
26810,26808,Black House Coffee Roasty Stout Beer,27,5,13.4
35309,35306,Gluten Free Organic Peanut Butter & Chocolate ...,121,14,6.8
35495,35491,Adore Forever Body Wash,127,11,9.9


In [27]:
#check to see number of rows 
df_prods_clean.shape

(49677, 5)

In [28]:
#remove full duplicates
df_prods_clean_no_dups = df_prods_clean.drop_duplicates()

In [29]:
#check to see that there are 5 fewer rows
df_prods_clean_no_dups.shape

(49672, 5)

## Export checked products

In [30]:
#export checked products
df_prods_clean_no_dups.to_csv(os.path.join(path, '02.Data', 'Prepared_Data', 'products_checked.csv'))

# Task Consistency Checks

In [31]:
#look at descriptive stats for orders
df_ords.describe()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


### min order day of week 0 max 6 - no issues
### min order hour of day 0 max 23 - no issues
### min days since last order 0 max 30 - no issues
### unclear why order numbers only go from 1 to 100

In [32]:
#check for mixed data types
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

### no mixed type columns found

## Missing values check

In [33]:
#check for missing values
df_ords.isnull().sum()

order_id                      0
user_id                       0
order_number                  0
order_day_of_week             0
order_hour_of_day             0
days_since_last_order    206209
dtype: int64

In [34]:
#check total number of rows to see percent of missing values
df_ords.shape

(3421083, 6)

### 6.03% of rows in orders contain missing values from the days_since_last_order column
### this could be due to someone not ordering a second time or an entry error

In [37]:
#create subset of missing values to see if any patterns exist
df_ords_miss_val = df_ords[df_ords['days_since_last_order'].isnull() == True]
df_ords_miss_val.head(25)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,NaN
11,2168274,2,1,2,11,NaN
26,1374495,3,1,1,14,NaN
39,3343014,4,1,6,11,NaN
45,2717275,5,1,3,12,NaN
50,2086598,6,1,5,18,NaN
54,2565571,7,1,3,9,NaN
75,600894,8,1,6,0,NaN
79,280530,9,1,1,17,NaN
83,1224907,10,1,2,14,NaN


In [39]:
df_ords_miss_val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 206209 entries, 0 to 3421069
Data columns (total 6 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   order_id               206209 non-null  int64  
 1   user_id                206209 non-null  int64  
 2   order_number           206209 non-null  int64  
 3   order_day_of_week      206209 non-null  int64  
 4   order_hour_of_day      206209 non-null  int64  
 5   days_since_last_order  0 non-null       float64
dtypes: float64(1), int64(5)
memory usage: 11.0 MB


In [41]:
df_ords_miss_val.describe()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
count,2.062090e+05,206209.000000,206209.0,206209.000000,206209.000000,0.0
mean,1.708462e+06,103105.000000,1.0,2.754118,13.626597,NaN
std,9.881299e+05,59527.555167,0.0,2.076205,4.223769,NaN
min,2.000000e+01,1.000000,1.0,0.000000,0.000000,NaN
25%,8.507300e+05,51553.000000,1.0,1.000000,11.000000,NaN
50%,1.706246e+06,103105.000000,1.0,3.000000,14.000000,NaN
75%,2.564292e+06,154657.000000,1.0,5.000000,17.000000,NaN
max,3.421081e+06,206209.000000,1.0,6.000000,23.000000,NaN


### all missing values on days_since_last_order also have order_number = 1
### check to see if there are the same number of order_number = 1 as there are missing values

In [43]:
df_ord_num1 = df_ords[df_ords['order_number'] == 1]
df_ord_num1.shape

(206209, 6)

In [ ]:
#learned in 4.10 you can use crosstab to conduct this investigation
missing_val_xtab = pd.crosstab(df_ords['days_since_last_order'], df_ords['order_number'], dropna = False)

### all orders with order number = 1 have missing days_since_last_order

In [54]:
#impute missing values with mean of 11 days 
df_ords['days_since_last_order'].fillna(11, inplace=True)

In [55]:
#check to see if missing values were addressed
df_ords.isnull().sum()

order_id                 0
user_id                  0
order_number             0
order_day_of_week        0
order_hour_of_day        0
days_since_last_order    0
dtype: int64

In [57]:
#check to see if missing values filled correctly 
df_ords[df_ords['order_number'] == 1]

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order
0,2539329,1,1,2,8,11.0
11,2168274,2,1,2,11,11.0
26,1374495,3,1,1,14,11.0
39,3343014,4,1,6,11,11.0
45,2717275,5,1,3,12,11.0
...,...,...,...,...,...,...
3420930,969311,206205,1,4,12,11.0
3420934,3189322,206206,1,3,18,11.0
3421002,2166133,206207,1,6,19,11.0
3421019,2227043,206208,1,1,15,11.0


### imputed missing values with mean of 11 days because more than 5% of data was missing. Using zero would imply that they ordered again the same day. would verify if order_number = 1 is a place holder or special in another way before imputing

## Look for duplicates in orders

In [58]:
df_ords_dups = df_ords[df_ords.duplicated()]
df_ords_dups

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order


In [59]:
#check for duplicates
df_ords_dups.shape

(0, 6)

### no full duplicates found in orders. 

## Export cleaned orders

In [61]:
df_ords.to_csv(os.path.join(path, 
                            '02.Data', 
                            'Prepared_Data', 
                            'orders_checked.csv'))